In [2]:
import boto3
import time
from datetime import datetime, timedelta
def run_query(query, database, s3_output):
    client = boto3.client('athena')
    response = client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
        },
        ResultConfiguration={
            'OutputLocation': s3_output,
        }
    )
    return response['QueryExecutionId']

def get_results(query_id):
    client = boto3.client('athena')
    
    while True:
        response = client.get_query_execution(QueryExecutionId=query_id)
        if response['QueryExecution']['Status']['State'] == 'SUCCEEDED':
            break
        elif response['QueryExecution']['Status']['State'] == 'FAILED':
            raise Exception("Athena query failed!")
        time.sleep(2)  # adjust this or use a back-off algorithm if needed
    
    result = client.get_query_results(QueryExecutionId=query_id)
    return result

In [6]:
# collect the tournament itself
def get_tournament(id):
    query = f"""
    WITH tourney AS (
        SELECT * 
        FROM lol.tournaments
        WHERE id = '{id}'
    )
    SELECT id, leagueid, name, startdate, enddate from tourney
    """
    database = "lol"
    s3_output = "s3://query-results-144/a/Dont-bill-me/"
    query_id = run_query(query, database, s3_output)
    result = get_results(query_id)
    tourney_info = []
    headers = []
    for i, Rows in enumerate(result['ResultSet']['Rows']):
        if i == 0:
            for El in Rows['Data']:
                val = El['VarCharValue']
                headers.append(val)
            continue
        tourney_info.append({f'{header}':Rows['Data'][i]['VarCharValue'] for i,header in enumerate(headers)})
    start_date = datetime.strptime(tourney_info[0]['startdate'], '%Y-%m-%d')
    six_months_prior = start_date - timedelta(days=6*30) 
    six_months_str = six_months_prior.strftime('%Y-%m-%d')
    tourney_info[0]['sixmonths'] = six_months_str
    return tourney_info[0]

In [26]:
# collect the tournament games
def get_tournament_matches(id):
    query = f"""
    WITH tourney AS (
        SELECT * 
        FROM lol.tournaments
        WHERE id = '{id}'
    ),
    tourney_matches AS (
        SELECT 
            t.*,
            stage.name AS stage_name,
            stage.type AS stage_type,
            stage.slug AS stage_slug,
            section.name AS section_name,
            match_item.id AS match_id,
            match_item.type AS match_type,
            match_item.state AS match_state,
            match_item.mode AS match_mode,
            match_item.strategy.type AS match_strategy_type,
            match_item.strategy.count AS match_strategy_count,
            team.id AS team_id,
            team.side AS team_side,
            team.record.wins AS team_wins,
            team.record.losses AS team_losses,
            team.record.ties AS team_ties,
            team.result.outcome AS team_outcome,
            team.result.gamewins AS team_gamewins,
            player.id AS player_id,
            player.role AS player_role
        FROM tourney
        CROSS JOIN UNNEST(stages) AS t (stage)
        CROSS JOIN UNNEST(stage.sections) AS s (section)
        CROSS JOIN UNNEST(section.matches) AS m (match_item)
        CROSS JOIN UNNEST(match_item.teams) AS tm (team)
        CROSS JOIN UNNEST(team.players) AS p (player)
    )
    SELECT distinct team_id, name, acronym, team_wins, team_losses, team_ties, team_gamewins 
    FROM tourney_matches
    JOIN lol.teams 
    USING(team_id)
    """
    database = "lol"
    s3_output = "s3://query-results-144/a/Dont-bill-me/"
    query_id = run_query(query, database, s3_output)
    result = get_results(query_id)
    tourney_matches = []
    headers = []
    for i, Rows in enumerate(result['ResultSet']['Rows']):
        if i == 0:
            for El in Rows['Data']:
                val = El['VarCharValue']
                headers.append(val)
            continue

        tourney_matches.append({f'{header}':Rows['Data'][i]['VarCharValue'] for i,header in enumerate(headers)})
    return tourney_matches

In [78]:


start_date = '2022-01-01'
def recent_game_stats(team_ids, start_date, end_date=None, days=182):
    
    if isinstance(team_ids, (list, tuple)):
        team_ids_str = ', '.join(map(str, team_ids))
    else:
        team_ids_str = str(team_ids)
    
    start_date_obj = datetime.strptime(start_date, '%Y-%m-%d')
    
    six_months_prior_obj = start_date_obj - timedelta(days=days)
    six_months_prior_str = six_months_prior_obj.strftime('%Y-%m-%d')
    
    query = f"""

WITH unnested_tournaments AS (
    SELECT 
        id AS league_id,
        region,
        tournament.id AS tournament_id
    FROM 
        lol.leagues
        CROSS JOIN UNNEST(tournaments) AS t (tournament)
),
tourney AS (
    SELECT * 
    FROM lol.tournaments
    WHERE startdate > '{six_months_prior_str}'
    AND startdate < '{start_date}'
),
tourney_matches AS (
    SELECT 
        t.*,
        tr.region,  -- Adding the region column here
        stage.name AS stage_name,
        stage.type AS stage_type,
        stage.slug AS stage_slug,
        section.name AS section_name,
        match_item.id AS match_id,
        match_item.type AS match_type,
        match_item.state AS match_state,
        match_item.mode AS match_mode,
        match_item.strategy.type AS match_strategy_type,
        match_item.strategy.count AS match_strategy_count,
        team.id AS team_id,
        team.side AS team_side,
        team.record.wins AS team_wins,
        team.record.losses AS team_losses,
        team.record.ties AS team_ties,
        team.result.outcome AS team_outcome,
        team.result.gamewins AS team_gamewins,
        player.id AS player_id,
        player.role AS player_role
    FROM tourney t
    JOIN unnested_tournaments tr ON tr.tournament_id = t.id  -- Joining on the tournament_id to get the region
    CROSS JOIN UNNEST(stages) AS t (stage)
    CROSS JOIN UNNEST(stage.sections) AS s (section)
    CROSS JOIN UNNEST(section.matches) AS m (match_item)
    CROSS JOIN UNNEST(match_item.teams) AS tm (team)
    CROSS JOIN UNNEST(team.players) AS p (player)
),
teamWins AS (
    SELECT distinct team_id, tourney_matches.region, lol.teams.name, acronym, team_wins, team_losses, team_ties, team_gamewins 
    FROM tourney_matches
    JOIN lol.teams 
    USING(team_id)
),
teamStats AS (
    select region, teamWins.name, teamWins.acronym, team_id, sum(team_wins) as nwin, sum(team_losses) nloss
    from teamWins
    join lol.teams using(team_id)
    group by region, team_id, teamWins.name, teamWins.acronym
)
SELECT * FROM teamStats


    """
    database = "lol"
    s3_output = "s3://query-results-144/a/Dont-bill-me/"
    query_id = run_query(query, database, s3_output)
    result = get_results(query_id)
    tourney_info = []
    headers = []
    for i, Rows in enumerate(result['ResultSet']['Rows']):
        if i == 0:
            for El in Rows['Data']:
                val = El['VarCharValue']
                headers.append(val)
            continue
        tourney_info.append({f'{header}':Rows['Data'][i]['VarCharValue'] for i,header in enumerate(headers)})
    return tourney_info




In [79]:


start_date = '2022-01-01'
def league_comparison(start_date, end_date=None, days=182):
    
    start_date_obj = datetime.strptime(start_date, '%Y-%m-%d')
    
    six_months_prior_obj = start_date_obj - timedelta(days=days)
    six_months_prior_str = six_months_prior_obj.strftime('%Y-%m-%d')
    
    query = f"""

WITH unnested_tournaments AS (
    SELECT 
        id AS league_id,
        region,
        tournament.id AS tournament_id
    FROM 
        lol.leagues
        CROSS JOIN UNNEST(tournaments) AS t (tournament)
),
tourney AS (
    SELECT * 
    FROM lol.tournaments
    WHERE startdate > '{six_months_prior_str}'
    AND startdate < '{start_date}'
),
tourney_matches AS (
    SELECT 
        t.*,
        tr.region,  -- Adding the region column here
        stage.name AS stage_name,
        stage.type AS stage_type,
        stage.slug AS stage_slug,
        section.name AS section_name,
        match_item.id AS match_id,
        match_item.type AS match_type,
        match_item.state AS match_state,
        match_item.mode AS match_mode,
        match_item.strategy.type AS match_strategy_type,
        match_item.strategy.count AS match_strategy_count,
        team.id AS team_id,
        team.side AS team_side,
        team.record.wins AS team_wins,
        team.record.losses AS team_losses,
        team.record.ties AS team_ties,
        team.result.outcome AS team_outcome,
        team.result.gamewins AS team_gamewins,
        player.id AS player_id,
        player.role AS player_role
    FROM tourney t
    JOIN unnested_tournaments tr ON tr.tournament_id = t.id  -- Joining on the tournament_id to get the region
    CROSS JOIN UNNEST(stages) AS t (stage)
    CROSS JOIN UNNEST(stage.sections) AS s (section)
    CROSS JOIN UNNEST(section.matches) AS m (match_item)
    CROSS JOIN UNNEST(match_item.teams) AS tm (team)
    CROSS JOIN UNNEST(team.players) AS p (player)
),
region_game_count AS (
    SELECT 
        region,
        COUNT(DISTINCT match_id) AS games_played
    FROM tourney_matches
    GROUP BY region
)

SELECT * FROM region_game_count
    """
    database = "lol"
    s3_output = "s3://query-results-144/a/Dont-bill-me/"
    query_id = run_query(query, database, s3_output)
    result = get_results(query_id)
    tourney_info = []
    headers = []
    for i, Rows in enumerate(result['ResultSet']['Rows']):
        if i == 0:
            for El in Rows['Data']:
                val = El['VarCharValue']
                headers.append(val)
            continue
        tourney_info.append({f'{header}':Rows['Data'][i]['VarCharValue'] for i,header in enumerate(headers)})
    return tourney_info

In [112]:
def process_tourney(id):
    tourney = get_tournament(id)
    matches = get_tournament_matches(id)

    start_date = tourney['startdate']

    lc = league_comparison(start_date)

    teams = [x['team_id'] for x in matches]

    team_data = recent_game_stats(teams, start_date)

    df = pd.DataFrame(team_data)

    df['nwin'] = df['nwin'].astype(int)
    df['nloss'] = df['nloss'].astype(int)

    df['win_loss_ratio'] = df.apply(lambda row: row['nwin'] if row['nloss'] == 0 else row['nwin'] / (row['nwin'] + row['nloss']), axis=1)

    df['ntot'] = df['nwin'] + df['nloss']

    threshold = 10

    filtered_df = df[df['ntot'] >= threshold]

    filtered_df_sorted = filtered_df.sort_values(by=['win_loss_ratio', 'nwin'], ascending=[False, False])

    filtered_df_sorted.reset_index(drop=True, inplace=True)

    return filtered_df

In [120]:
df = process_tourney(106342376604772906)

In [122]:
df

,region,name,acronym,team_id,nwin,nloss,win_loss_ratio,ntot
0,EMEA,JD|XL,JDXL,102787200155434012,51,25,0.671053,76
1,EMEA,Nordavind,NVD,105519977854458717,34,29,0.539683,63
2,EMEA,Anc Outplayed,OP,105537149834356301,34,34,0.500000,68
3,EMEA,Macko Esports,MCK,105536918475957734,37,18,0.672727,55
4,EMEA,PENTA 1860,PTA,105503848339138647,40,33,0.547945,73
...,...,...,...,...,...,...,...,...
214,EMEA,CR4ZY,C4,104710373370426042,20,11,0.645161,31
216,EMEA,MAD Lions,MAD,103461966965149786,23,16,0.589744,39
217,EMEA,Team Vitality,VIT,99322214695067838,10,26,0.277778,36
218,EMEA,ASUS ROG Elite,ASUS,102787200130988976,4,26,0.133333,30


In [114]:
# get_tournament_matches(106342376604772906)

In [ ]:
# league_comparison('2023-12-12', days=79990)

In [ ]:
# team_data = recent_game_stats(['98767991954244555', '103877625775457850', '104367068120825486', '105913111502565010'], '2023-12-12', days=79990)